In [1]:
import faiss
import pandas as pd
import numpy as np
from openai import AzureOpenAI

client = AzureOpenAI(
    api_key="YOUR_AZURE_OPENAI_KEY",
    api_version="2024-05-01-preview",
    azure_endpoint="https://team05-openai.openai.azure.com"
)

# 1. 메타데이터 & FAISS 인덱스 로드
df_meta = pd.read_csv("FAISS/faiss_metadata.csv", encoding="utf-8-sig")
index = faiss.read_index("FAISS/faiss_index.idx")

# 2. 사용자 질문 → 임베딩
query = "열이 나서 병원에 입원했는데, 보험금 청구가 가능한가요?"
resp = client.embeddings.create(
    input=[query],
    model="text-embedding-3-large"
)
query_vec = np.array(resp.data[0].embedding, dtype='float32').reshape(1, -1)

# 3. 유사 조항 top-3 검색
D, I = index.search(query_vec, k=3) 

# 4. 유사 조항 출력
context = "\n\n".join(df_meta.loc[i, "chunk"] for i in I[0])

# 5. GPT 응답 생성
completion = client.chat.completions.create(
    model="gpt-4",
    messages=[
        {"role": "system", "content": "당신은 보험 약관 해석 전문 AI 챗봇입니다. 다음 문서 조항을 기반으로 질문에 답하고, 보기 쉽게 불렛 포인트로 조항을 보여줘. 친절한 상담사처럼 설명해주고 전문용어를 쉽게 풀어줘. 파일에 없는 내용은 답변하지 마세요."},
        {"role": "user", "content": f"문서:\n{context}\n\n질문: {query}"}
    ]
)

print("질문:", query)
print("🔹 GPT 응답:\n", completion.choices[0].message.content)


질문: 열이 나서 병원에 입원했는데, 보험금 청구가 가능한가요?
🔹 GPT 응답:
 각 문서의 조항에 따라 보험금 청구 가능 여부를 확인하기 위해 입원 정의 및 보장 조건을 살펴보겠습니다.

1. **제3조 (입원의 정의와 장소):**
   - '입원'은 의료법 제3조에서 규정한 국내 병원, 의원, 또는 국외 법에서 정한 의료기관에 입실하여 의사의 관리 하에 전념하여 치료하는 것입니다.
   - 질병에서 회복을 위해 자택 등에서의 치료가 어려워 병원 등에 입실 필요한 경우를 의미합니다.

2. **병원 실제 입원 조건들:**
   - 입원실에 머무른 시간이 6시간 이상일 경우, 입원으로 간주할 수 있습니다.

3. **간병인 사용 관련 조항:**
   - 상해로 입원하는 경우에만 간병인 사용에 대한 보장이 적용되며 요양병원 및 의원에서의 입원은 제외됩니다(상해입원간병인사용급여금).

결론적으로, 귀하가 열이 나서 입원한 경우 보험금 청구 가능 여부는 다음과 같습니다:
- 단순히 '열이 났다'는 이유만으로는 상세한 진단 결과나 추가적인 의료상황이 명시되어 있지 않기 때문에, 해당 조건이 '질병에서 회복을 위해 병원 치료가 필요한 상태'에 부합하는지가 관건입니다.
- 입원으로 인정받기 위해서는 최소한 6시간 이상 병원에 머무는 것이 요구됩니다.
- 이 문서에서는 "간병인 사용 급여"는 상해로 인한 경우에만 적용되므로, 귀하의 경우 (질병 관련)는 해당 사항이 적용되지 않습니다.

이러한 조건들을 충족한다면, 보험금 청구가 가능할 수 있습니다. 보험 약관에 따른 정확한 조건 충족 여부를 최종 판단하기 위해, 의료 기록과 함께 보험 회사에 문의하시기를 추천드립니다.
